In [1]:
import sys
sys.path.append('../scripts') 

In [2]:
from apikey import nemo_apikey, openai_apikey

In [3]:
import os
os.environ['OPENAI_API_KEY']=openai_apikey

In [4]:
import openai

In [5]:
openai.api_key = openai_apikey

In [6]:
import requests
from bs4 import BeautifulSoup


url = "https://en.wikipedia.org/wiki/GPT-4"
response = requests.get(url)


soup = BeautifulSoup(response.content, 'html.parser')


# find the content div
content_div = soup.find('div', {'class': 'mw-parser-output'})


# remove unwanted elements from div
unwanted_tags = ['sup', 'span', 'table', 'ul', 'ol']
for tag in unwanted_tags:
    for match in content_div.findAll(tag):
        match.extract()


print(content_div.get_text())

2023 text-generating language model
"ChatGPT-4" redirects here. For other uses, see GPT.




Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its numbered "GPT-n" series of GPT foundation models. It was released on March 14, 2023, and has been made publicly available in a limited form via the chatbot product ChatGPT Plus (a premium version of ChatGPT), and with access to the GPT-4 based version of OpenAI's API being provided via a waitlist. As a transformer based model, GPT-4 was pretrained to predict the next token (using both public data and "data licensed from third-party providers"), and was then fine-tuned with reinforcement learning from human and AI feedback for human alignment and policy compliance.
Observers reported the GPT-4 based version of ChatGPT to be an improvement on the previous (GPT-3.5 based) ChatGPT, with the caveat that GPT-4 retains some of the same problems. Unlike the predecessors, GPT-4 can 

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


article_text = content_div.get_text()


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 200,
    chunk_overlap  = 20,
    length_function = len,
)


texts = text_splitter.create_documents([article_text],[{'source':'wiki'}])
print(texts[0])
print(texts[1])

page_content='2023 text-generating language model\n"ChatGPT-4" redirects here. For other uses, see GPT.' metadata={'source': 'wiki'}
page_content='Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its numbered "GPT-n" series of GPT foundation models. It was released on March' metadata={'source': 'wiki'}


In [8]:
texts[0].metadata

{'source': 'wiki'}

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# define the embeddings model
embeddings = OpenAIEmbeddings()

# use the text chunks and the embeddings model to fill our vector store
db = Chroma.from_documents(texts, embeddings)

In [10]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

users_question = "What is GPT-4?"

# use our vector store to find similar text chunks
results = db.similarity_search(
    query=users_question,
    n_results=5
)

# define the prompt template
template = """
You are a chat bot who loves to help people! Given the following context sections, answer the
question using only the given context. If you are unsure and the answer is not
explicitly writting in the documentation, say "Sorry, I don't know how to help with that."

Context sections:
{context}

Question:
{users_question}

Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "users_question"])

# fill the prompt template
prompt_text = prompt.format(context = results, users_question = users_question)


In [11]:
for r in results:
    print(r)

page_content='GPT-4 was trained in two stages. First, the model was given large datasets of text taken from the internet and trained to predict the next token (roughly corresponding to a word) in those datasets.' metadata={'source': 'wiki'}
page_content='Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its numbered "GPT-n" series of GPT foundation models. It was released on March' metadata={'source': 'wiki'}
page_content='2023 text-generating language model\n"ChatGPT-4" redirects here. For other uses, see GPT.' metadata={'source': 'wiki'}
page_content='Microsoft Research tested the model behind GPT-4 and concluded that "it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system".' metadata={'source': 'wiki'}


In [12]:
r.page_content

'Microsoft Research tested the model behind GPT-4 and concluded that "it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system".'

In [13]:
print(prompt_text)


You are a chat bot who loves to help people! Given the following context sections, answer the
question using only the given context. If you are unsure and the answer is not
explicitly writting in the documentation, say "Sorry, I don't know how to help with that."

Context sections:
[Document(page_content='GPT-4 was trained in two stages. First, the model was given large datasets of text taken from the internet and trained to predict the next token (roughly corresponding to a word) in those datasets.', metadata={'source': 'wiki'}), Document(page_content='Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its numbered "GPT-n" series of GPT foundation models. It was released on March', metadata={'source': 'wiki'}), Document(page_content='2023 text-generating language model\n"ChatGPT-4" redirects here. For other uses, see GPT.', metadata={'source': 'wiki'}), Document(page_content='Microsoft Research tested the model behin

In [14]:
from nemollm.api import NemoLLM

# Basic instantiation
conn = NemoLLM(api_key=nemo_apikey)

# Text completion
response = conn.generate(model="gpt-43b-001", prompt=prompt_text)
print(response)

/opt/conda/envs/rapids/lib/python3.10/site-packages/nemollm/api.py:79: UserWarning: ORG ID is not set. Organization id is required if you are in multiple LLM enabled organizations. org_id can be set when instantiating NemoLLM or set with environment variable 'export NGC_ORG_ID=<your_ngc_org_id>'
  warnings.warn(


{'text': " Sorry, I don't know how to help with that.", 'cumlogprobs': -0.8152105, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996882}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.992423}]}


In [15]:
prompt = """
You are a chat bot who loves to help people! Given the following context sections, answer the
question using only the given context. If you are unsure and the answer is not
explicitly writting in the documentation, say "Sorry, I don't know how to help with that."
"""

In [16]:
context = 'Context:\n' + '\n'.join([r.page_content for r in results])
print(context)

Context:
GPT-4 was trained in two stages. First, the model was given large datasets of text taken from the internet and trained to predict the next token (roughly corresponding to a word) in those datasets.
Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its numbered "GPT-n" series of GPT foundation models. It was released on March
2023 text-generating language model
"ChatGPT-4" redirects here. For other uses, see GPT.
Microsoft Research tested the model behind GPT-4 and concluded that "it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system".


In [17]:
question = '\n Question:\n' + users_question

In [18]:
prompt = '\n'.join([prompt, context, question, '\n Answer:\n'])
print(prompt)


You are a chat bot who loves to help people! Given the following context sections, answer the
question using only the given context. If you are unsure and the answer is not
explicitly writting in the documentation, say "Sorry, I don't know how to help with that."

Context:
GPT-4 was trained in two stages. First, the model was given large datasets of text taken from the internet and trained to predict the next token (roughly corresponding to a word) in those datasets.
Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its numbered "GPT-n" series of GPT foundation models. It was released on March
2023 text-generating language model
"ChatGPT-4" redirects here. For other uses, see GPT.
Microsoft Research tested the model behind GPT-4 and concluded that "it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system".

 Question:
What is GPT-4?

 Answer:



In [19]:
# Text completion
response = conn.generate(model="gpt-43b-001", prompt=prompt)
print(response)

{'text': '', 'cumlogprobs': -0.9859122, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.532528}]}


In [22]:
# Text completion
for s in range(10):
    response = conn.generate(model="gpt-43b-001", prompt=prompt,top_k=10,random_seed=s)
    print(response)

{'text': ' ChatGPT-4', 'cumlogprobs': -3.0005655, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'toxic', 'score': 0.672923}]}
{'text': ' text-generating language model', 'cumlogprobs': -1.6152089, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.987568}]}
{'text': '', 'cumlogprobs': -0.9859122, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.532528}]}
{'text': ' "ChatGPT-4" redirects here. For other uses, see GPT.', 'cumlogprobs': -4.149778, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.942295}]}
{'text': '', 'cumlogprobs': -0.9859122, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.532528}]}
{'text': '', 'cumlogprobs': -0

In [26]:
# Text completion
response = conn.generate(model="gpt-43b-002", prompt=prompt)
print(response)

{'text': ' GPT-4 is a large language model created by OpenAI.', 'cumlogprobs': -3.1834264, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.984276}]}


In [27]:
# Text completion
for s in range(10):
    response = conn.generate(model="gpt-43b-002", prompt=prompt,top_k=10,random_seed=s)
    print(response)

{'text': ' GPT-4 is a large language model created by OpenAI that was released in March 2023. The', 'cumlogprobs': -11.125062, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.982356}]}
{'text': ' GPT-4 is a large language model created by OpenAI, and the fourth in its numbered "GPT-n', 'cumlogprobs': -3.955577, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.637059}]}
{'text': ' GPT-4 is a large language model.', 'cumlogprobs': -3.4501226, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.977392}]}
{'text': ' A large language model developed by OpenAI.', 'cumlogprobs': -8.969741, 'prompt_labels': [{'class_name': 'nontoxic', 'score': 0.996385}], 'completion_labels': [{'class_name': 'nontoxic', 'score': 0.995549}]}
{'text': ' GPT-4 is a Large Language Model.',

In [25]:
messages=[{"role": "user", "content": prompt}]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    max_tokens=100,
    temperature=1.2,
    messages = messages)

print(response)

{
  "warning": "This model version is deprecated and a newer version 'gpt-3.5-turbo-0613' is available. Migrate before September 13 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "chatcmpl-7bfMPE1lTcmigLJq7ON0hrrGIafKb",
  "object": "chat.completion",
  "created": 1689211481,
  "model": "gpt-3.5-turbo-0301",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "GPT-4 is a multimodal large language model created by OpenAI and the fourth in its numbered \"GPT-n\" series of GPT foundation models. It is a text-generating language model. Microsoft Research tested the model behind GPT-4 and concluded that \"it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system\"."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 238,
    "completion_tokens": 81,
    "total_tokens": 319
  }
}
